In [94]:
import pandas as pd
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
lemma = WordNetLemmatizer()
# from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package wordnet to /home/depa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/depa/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


### Merchants data preprocessing

In [32]:
merchants_df = pd.read_parquet('../data/tables/tbl_merchants.parquet').reset_index()

In [33]:
# A sample of an entry in the tags columns of merchants data
merchants_df.head(1).loc[0, 'tags']

'((furniture, home furnishings and equipment shops, and manufacturers, except appliances), (e), (take rate: 0.18))'

In [34]:
#First separte text into 3 separate features (separated by () or [])
def separate_tags(row):
    features = re.findall(r'[\(|\[][\(|\[](.*)[\)|\]],\s[\(|\[](.*)[\)|\]],\s[\(|\[](.*)[\)|\]][\)|\]]', row['tags'])
    row['feature_1'] = features[0][0]
    row['feature_2'] = features[0][1]
    row['feature_3'] = features[0][2]
    return row
merchants_df = merchants_df.apply(separate_tags, axis = 1)

In [42]:
# convert the take rate feature (feature_2) to float type
# def get_take_rate(x):
#     feature = re.findall('take rate: (\d+\.\d+)', x)
#     return float(feature[0])

# merchants_df['take_rate'] = merchants_df['feature_3'].apply(get_take_rate)
merchants_df = merchants_df.rename(columns = {'feature_2': 'revenue_level', 'feature_1': 'category'})
merchants_df = merchants_df.drop(columns = ['tags', 'feature_3'])

In [76]:
merchants_df

,merchant_abn,name,category,revenue_level,take_rate
0,10023283211,Felis Limited,"furniture, home furnishings and equipment shop...",e,0.18
1,10142254217,Arcu Ac Orci Corporation,"cable, satellite, and otHer pay television and...",b,4.22
2,10165489824,Nunc Sed Company,"jewelry, watch, clock, and silverware shops",b,4.40
3,10187291046,Ultricies Dignissim Lacus Foundation,"wAtch, clock, and jewelry repair shops",b,3.29
4,10192359162,Enim Condimentum PC,"music shops - musical instruments, pianos, and...",a,6.33
...,...,...,...,...,...
4021,99938978285,Elit Dictum Eu Ltd,"opticians, optical goods, and eyeglasses",b,4.50
4022,99974311662,Mollis LLP,"books, periodicals, and newspapers",b,3.17
4023,99976658299,Sociosqu Corp.,shoe shops,a,6.57
4024,99987905597,Commodo Hendrerit LLC,motor vehicle Supplies and new parts,a,6.82


In [132]:
# 971 Unique instances of the unpreprocessed tags
# Preprocessing 'category' occurs in this cell

merchants_df['category'] = merchants_df['category'].str.lower()
merchants_df['category'] = merchants_df['category'].str.split(' and ')

new_category = []
for e in merchants_df['category']:
    new = ', '.join(e)
    new_category.append(new)
merchants_df['category'] = new_category

merchants_df['category'] = merchants_df['category'].str.split(',')

new_category = []
for e in merchants_df['category']: 
    # # Lemmatizing
    # for i in e:
    #     new_e.append(lemma.lemmatize(i))
    
    new_e = []
    # Removing leading and trailing whitespace
    for i in e:
        word = i.lstrip().rstrip()
        word = re.sub(r'\s{2,}', ' ', word)
        new_e.append(word)

    # Removing empty options in list
    removeIndex = []
    for i in range(len(new_e)):
        if not new_e[i]:
            removeIndex.insert(0, i)
    for i in removeIndex:
        new_e.pop(i)     

    new_category.append(new_e)

merchants_df['category'] = new_category

In [136]:
merged_categories = []
for e in merchants_df['category']:
    merged_categories.extend(e)
print(len(merged_categories))
C = Counter(merged_categories)
print(len(C))

11731
70


In [135]:
merchants_df

,merchant_abn,name,category,revenue_level,take_rate
0,10023283211,Felis Limited,"[furniture, home furnishings, equipment shops,...",e,0.18
1,10142254217,Arcu Ac Orci Corporation,"[cable, satellite, other pay television, radio...",b,4.22
2,10165489824,Nunc Sed Company,"[jewelry, watch, clock, silverware shops]",b,4.40
3,10187291046,Ultricies Dignissim Lacus Foundation,"[watch, clock, jewelry repair shops]",b,3.29
4,10192359162,Enim Condimentum PC,"[music shops - musical instruments, pianos, sh...",a,6.33
...,...,...,...,...,...
4021,99938978285,Elit Dictum Eu Ltd,"[opticians, optical goods, eyeglasses]",b,4.50
4022,99974311662,Mollis LLP,"[books, periodicals, newspapers]",b,3.17
4023,99976658299,Sociosqu Corp.,[shoe shops],a,6.57
4024,99987905597,Commodo Hendrerit LLC,"[motor vehicle supplies, new parts]",a,6.82
